# <strong>Aplicação de Sistemas Lineares na Estequiometria:</strong> Desenvolvimento de uma Ferramenta Computacional
#### Autores: Gabriel Viégas Ribeiro, Glauber Nascimento de Oliveira, Lorena Ribeiro Nascimento, Maria Emily Nayla Gomes

O balanceamento de equações químicas é uma etapa fundamental na química, pois é essencial para garantir que a lei da conservação da massa seja respeitada, ou seja, a quantidade de átomos de cada elemento químico deve ser a mesma nos reagentes e nos produtos. Este trabalho apresenta o desenvolvimento de uma calculadora estequiométrica baseada em métodos computacionais, com foco na aplicação do Método Iterativo de Gauss-Seidel, além da Eliminação Gaussiana, para resolver sistemas de equações lineares que representam os coeficientes estequiométricos de uma reação química. A ferramenta foi projetada para receber como entrada uma string no formato convencional de reações químicas (por exemplo, "Reagente1 + Reagente2 -> Produto1 + Produto2"), identificar os componentes da reação e, a partir disso, calcular e fornecer a equação balanceada como saída.

A escolha do Método Iterativo de Gauss-Seidel deve-se à sua capacidade de lidar com matrizes esparsas. Dessa forma, este trabalho busca não apenas criar uma solução computacional para o balanceamento químico, mas também destacar a importância dos métodos numéricos como ferramentas no contexto de problemas científicos.

 ### Importando as bibliotecas
Com a biblioteca <code>numpy</code>, é possível realizar operações de array, com a <code>re</code>, lidou-se com o input e se organizou a matriz com o valor de cada átomo presente em cada molécula da reação. Além disso, a <code>fractions</code> facilitou a transformação de coeficientes decimais em inteiros.</p>

In [1]:
import numpy as np
import re
from fractions import Fraction

## Métodos de resolução

### Método iterativo Gauss-Seidel
Temos três funções nesse método: a primeira, vai verificar se a matriz segue os critérios definidos por nós; a segunda, realiza um pivoteamento caso não esteja convergindo, e a terceira é de fato o método iterativo.<br>
Além dos critérios comuns de convergência (critério das linhas e não haver zeros não pivoteáveis na diagonal principal), acrescentou-se o critério da matriz ser no mínimo 3x3. Esse critério surge pela natureza das matrizes correspondentes a coeficientes estequiométricos: em geral, aparecem zeros na diagonal secundária que atrapalham a resolução por métodos numéricos. Assim, todas as matrizes 2x2 são resolvidas pelo método direto.

In [2]:
def criterios(A,b, iterr=100):
    """
    Função que verifica se o sistema cumpre os critérios para ser resolvido por método numérico

    Args: 
        A (numpy.ndarray): Matriz com coeficientes. Deve ser numérica 
            (dtype float ou int).
        b (numpy.ndarray): Vetor de termos independentes. Deve ser numérico 
            (dtype float ou int).
        iterr (int, optional): Número máximo de iterações de pivoteamento. O padrão é 100.

    Returns:
        numpy.ndarray or bool: A matriz A e o vetor b (pivoteados ou não) se cumprirem todos os critérios, ou False se não.

    """

    if A.shape[0] < 3:
        return False # o método direto lida melhor com matrizes pequenas
    
    while (np.any(np.diag(A) == 0) == True):

        for i in range(iterr):
            A, b = pivoteamento(A,b)
            
        break
    
    if np.any(np.diag(A) == 0) == True:
        return False
    
    def convergencia(A):
        """Verifica se a matriz A cumpre o critério das linhas para convergência de matrizes resolvidas pelo método numérico."""

        linhas, colunas = A.shape
    
        for i in range(linhas):
        
            soma = 0
            for j in range(colunas):
                if j == i:
                    pass
                else:
                    soma = soma + A[i,j]
            divisao = soma / A[i,i]

            if divisao > 1:
                return False
        return True
    
    if convergencia(A) == False:
        return False
    
    return A,b

def pivoteamento(A,b):
    """
    Função que realiza o método de pivoteamento para um sistema linear formado por uma matriz A e um vetor resposta b

    Args: 
        A (numpy.ndarray): Matriz com coeficientes. Deve ser quadrada e numérica 
            (dtype float ou int).
        b (numpy.ndarray): Vetor de termos independentes. Deve ser numérico 
            (dtype float ou int).

    Returns:
        tuple: Tupla contendo:
            - A (numpy.ndarray): matriz A pivoteada
            - b (numpy.ndarray): vetor b pivoteado
            
    """

    linhas = A.shape[0]
    for j in range(linhas): # colunas
        
        # Achar o index do maior pivô
        index_maior_a = np.argmax(np.abs(A[j:, j])) + j
        A[[j, index_maior_a]] = A[[index_maior_a, j]]
        b[[j, index_maior_a]] = b[[index_maior_a, j]]
        
    return A,b

def gauss_seidel(A, b, chute=None, iterr=100, tol=0.05):
    """
    Resolve sistemas lineares utilizando o método de Gauss-Seidel.

    Args: 
        A (numpy.ndarray): Matriz com coeficientes. Deve ser numérica 
            (dtype float ou int).
        b (numpy.ndarray): Vetor de termos independentes. Deve ser numérico 
            (dtype float ou int).
        chute (numpy.ndarray, optional): Vetor de chute. Deve ser numérico
            (dtype float ou int) ou None. O padrão é um vetor de uns.
        iterr (int, optional): Número máximo de iterações de pivoteamento. O padrão é 100.
        tol (float, optional): Diferença mínima entre o vetor solução anterior e o iterado. 
            Representa o critério de convergência. O padrão é 5%.

    Returns:
        numpy.ndarray: Solução aproximada do sistema em forma de vetor.

    Raises:
        ValueError: Se a matriz não aderir aos critérios de Gauss-Seidel.

    """

    if chute==None:
        chute = np.ones(b.shape)

    if criterios(A,b, iterr) == False:
        raise ValueError('A matriz não adere aos critérios de Gauss-Seidel.')
    else:
        A, b = criterios(A,b, iterr)
    
    for _ in range(iterr):

        # iteração de chutes
        chute_antes = chute
        
        for i in range(len(b)):
            soma = sum(A[i, j] * chute[j] for j in range((len(b))) if j != i)
            chute[i] = (b[i] - soma) / A[i, i] # chute atualizado a cada iteração
        
        # erro relativo
        distancia = max(abs(chute-chute_antes))
        error = distancia/(max(abs(chute)))

        if error <= tol:
            break

    return chute

### Método direto Eliminação Gaussiana
Se a matriz não cumprir os critérios para a resolução por métodos numéricos, utilizaremos o método direto de eliminação gaussiana, que se utiliza da função de resolução triangular para retornar o resultado, como descrito abaixo:

In [3]:
def resolucao_triangular(A, b):
    """
    Resolve sistemas lineares em que a matriz A é triangular superior utilizando substituição direta.

    Args: 
        A (numpy.ndarray): Matriz com coeficientes. Deve ser triangular e numérica 
            (dtype float ou int).
        b (numpy.ndarray): Vetor de termos independentes. Deve ser numérico 
            (dtype float ou int).

    Returns:
        numpy.ndarray: Solução do sistema em forma de vetor.

    Raises:
        ValueError: Se a matriz A não for quadrada ou houver um zero na diagonal.

    """ 
    
    linhas, colunas = A.shape
    resolucao = np.zeros(colunas)
    
    if linhas != colunas:
        raise ValueError('Essa não é uma matriz quadrada :| Esse não é o melhor método!')

    else:
        # Começa de cima para baixo na matriz
        for i in range(linhas - 1, -1, -1):
            if A[i, i] == 0:
                raise ValueError('Há um zero na diagonal! Não dá pra resolver :(')
                return
            else:
                nova_linha = np.sum(A[i, i+1:] * resolucao[i+1:]) # faz as operações na linha, com os multiplicadores etc
                resolucao[i] = (b[i] - nova_linha) / A[i, i]
    
    return resolucao

def eliminacao_gauss_pivo(A,b):
    """
    Resolve sistemas lineares pelo método de eliminação gaussiana com pivoteamento.

    Args: 
        A (numpy.ndarray): Matriz com coeficientes. Deve ser triangular e numérica 
            (dtype float ou int).
        b (numpy.ndarray): Vetor de termos independentes. Deve ser numérico 
            (dtype float ou int).

    Returns:
        numpy.ndarray: Solução do sistema em forma de vetor.
    
    Raises:
        ValueError: Se a matriz A não for quadrada ou houver um zero na diagonal.

    """

    # Verifica o número de linhas da matriz.
    linhas = A.shape[0]
    
    for j in range(linhas): # colunas
        
        # Achar o index do maior pivô
        index_maior_a = np.argmax(np.abs(A[j:, j])) + j
        A[[j, index_maior_a]] = A[[index_maior_a, j]]
        b[[j, index_maior_a]] = b[[index_maior_a, j]]
        
        for i in range(j+1, linhas): # linhas
            
            multiplicador = A[i,j] / A[j,j]
            
            for k in range(j, linhas):
                A[i,k] -= multiplicador * A[j,k]
                
            b[i] -= multiplicador * b[j]

    resposta = resolucao_triangular(A,b)
        
    return resposta

## Aplicação

### Código de tratamento e cálculo
Teremos um DataFrame com os valores de A organizados de maneira que devemos passar os valores para um mesmo lado. O vetor resposta 'b' será sempre 0, uma vez que não há valores inteiros não acompanhados de variáveis no nosso caso.

In [4]:
def tratamento(A,b):
    """
    Tratamento da matriz com coeficientes e o vetor de termos independentes. Transforma matrizes retangulares em quadradas.

    A função identifica a coluna de `A` com o maior número de zeros e a remove, ajustando o vetor `b` subtraindo a linha correspondente da coluna 
    removida. A resposta da coluna removida é igualada a 1.

    Args: 
        A (numpy.ndarray): Matriz com coeficientes. Deve ser triangular e numérica 
            (dtype float ou int).
        b (numpy.ndarray): Vetor de termos independentes. Deve ser numérico 
            (dtype float ou int).

    Returns:
        tuple: Tupla contendo:
            - A (numpy.ndarray): matriz A tratada (quadrada)
            - b (numpy.ndarray): vetor b tratado (conforme a coluna retirada)
            - j (int): índice da coluna retirada 

    """

    transposta = A.T
    qtd_zeros = []

    for i in transposta:
        qtd_zeros.append((i == 0).sum()) # descobre a coluna com mais zeros
    
    j = qtd_zeros.index(max(qtd_zeros)) #coluna que será igualada a 1 (e retirada)

    A = (np.delete(transposta, j, axis=0)).T
    b = b - transposta[j]

    return A, b, j

def calculo(A,b, formula):
    """
    Realiza o cálculo da solução de um sistema linear utilizando o método de Gauss-Seidel ou, em caso de erro, o método de Eliminação de Gauss com pivoteamento.

    A função verifica se a matriz `A` é quadrada. Se não for, a função `tratamento` é chamada para ajustar `A` e `b`. 
    Em seguida, o método de Gauss-Seidel é tentado para resolver o sistema. Caso a matriz não cumpra os critérios para ser resolvida por Gauss-Seidel,
    a eliminação de Gauss com pivoteamento é utilizada como alternativa. Após calcular a solução, a função tenta tornar os resultados inteiros, 
    multiplicando por um denominador comum.

    Args:
        A (numpy.ndarray): Matriz com coeficientes. Deve ser numérica 
            (dtype float ou int).
        b (numpy.ndarray): Vetor de termos independentes. Deve ser numérico 
            (dtype float ou int).

    Returns:
        numpy.ndarray: Vetor solução ajustado, com todos os valores convertidos para inteiros, e com o valor 1 inserido, se a matriz inicial não era quadrada.
    
    Notes:
        - Se a matriz A não for quadrada, ela é ajustada pelo método tratamento, que pode remover uma coluna de A e ajustar b.
        - A solução final é multiplicada por um denominador comum, se houver frações, para tornar os valores inteiros.

    """

    if A.shape[0] != A.shape[1]:
        matriz_tratada, resposta_tratada, j = tratamento(A,b)
    else:
        matriz_tratada, resposta_tratada = A,b

    try:
        r = gauss_seidel(matriz_tratada, resposta_tratada)
    except ValueError:
        # print('não foi numérico')
        r = eliminacao_gauss_pivo(matriz_tratada,resposta_tratada)
    
    if A.shape[0] != A.shape[1]:
        r = np.insert(r, j, 1)

    while np.any([not numero.is_integer() for numero in r]):
        denominador = 1
        for _, numero in enumerate(r):
            fracao = Fraction(numero).limit_denominator()
            
            if fracao.denominator > denominador:
                denominador = fracao.denominator

        r = r * denominador

    return expressando(r, formula)

### Funções para o input: Recebe as equações químicas

As funções desta seção do notebook são responsáveis por obter a equação química balanceada. O objetivo é permitir que o usuário insira uma reação contendo os reagentes e produtos, no formato: $Al + Cl_2 \rightarrow AlCl_3$. Esse formato deve ser rigorosamente respeitado para evitar erros no processamento. Será possível identificar o sinal de '+' como um separador de diferentes moléculas e usar o símbolo '->' como referência para separar reagentes de produtos. Além disso, cada molécula deve seguir a notação química padrão, utilizando letras maiúsculas e minúsculas para os símbolos dos elementos, seguidas de números opcionais para os índices estequiométricos. Caso não haja um número após o elemento, o programa interpretará que ele está presente apenas um desse átomo na molécula.<br>
O código interpreta essa entrada e extrai informações essenciais, como os tipos de átomos envolvidos e suas respectivas quantidades em cada molécula. Essas informações são utilizadas posteriormente para montar sistemas de equações lineares. Ao serem resolvidos pelo método de Gauss-Seidel, esses sistemas permitem determinar os coeficientes que balanceiam a reação química.

In [5]:
def encontra_atomos(moleculas):
    """
    Função que identifica todos os átomos únicos presentes nas moléculas fornecidas.

    Args:
        moleculas (list of str): Lista contendo strings que representam as moléculas dos reagentes.

    Returns:
        list of str: Lista, em ordem de ocorrência, dos átomos únicos encontrados nas moléculas.
    """
    atomos = []
    
    for molecula in moleculas:
        atomos.extend(re.findall(r"([A-Z][a-z]*)", molecula))

    for elemento, indice in zip(atomos[::-1], range(len(atomos)-1,0,-1)):
        if atomos.count(elemento) > 1:
            atomos.pop(indice)
    
    return atomos

In [6]:
def eosparenteses(molecula):
    """
    Função que lida com fórmulas químicas com parênteses, desse modo é possível considerar
    os multiplicadores fora dos parênteses.

    Args:
        molecula (str): String representando uma fórmula química.

    Returns:
        list of list: Lista contendo sublistas, onde cada sublista é formada pelo nome do átomo (str)
                      e sua quantidade que pode ser uma string (caso não tenha sido convertida) ou 
                      um número inteiro (int) se precisou achar o produto entre o coeficiente do átomo 
                      e o coeficiente do grupo.
    """
    aqui, indice = [], 0

    for i in range(len(molecula)):
        if molecula[i] == "(":
            indice = i
            aqui.extend(re.findall(r"([A-Z][a-z]*)(\d*)", molecula[:i]))
        elif molecula[i] == ")":
            valor = molecula[i+1]
            parenteses = re.findall(r"([A-Z][a-z]*)(\d*)", molecula[indice:i])
            parenteses = [list(match) for match in parenteses]
            
            for indice in range(len(parenteses)):
                if parenteses[indice][1] == "":
                    parenteses[indice][1] = molecula[i+1]
                else:
                    parenteses[indice][1] = int(parenteses[indice][1])*int(molecula[i+1])
                    
        
            aqui.extend(parenteses)    
            aqui.extend(re.findall(r"([A-Z][a-z]*)(\d*)", molecula[i+2:]))

    return aqui

In [7]:
def encontra_coef(atomos, moleculas, qtd_reagentes):
    """
    Função que calcula a matriz de coeficientes para um sistema linear representando
    o balanceamento de uma equação química em que as colunas representam as moléculas da
    reação e as linhas - os átomos, que foram encontrados em ordem de ocorrência.

    Args:
        atomos (list of str): Lista de átomos únicos presentes na reação química.
        moleculas (list of str): Lista de moléculas da equação química (reagentes e produtos).
        qtd_reagentes (int): Quantidade de moléculas que pertencem aos reagentes. 
                             Utilizado para ajustar os sinais dos coeficientes dos produtos, 
                             pois para o método interativo de Gauss-Seide é necessário.

    Returns:
        numpy.ndarray: Matriz de coeficientes em que as linhas representam os átomos e as
                       colunas - as molécula. Os valores indicam as quantidades de cada átomo
                       presente em cada molécula. Para os produtos, os coeficientes são negativos.
    """
    coef = np.array([[0]*len(moleculas)]*len(atomos))
    
    for _, coluna in zip(moleculas, range(len(moleculas))):
        dados =  eosparenteses(moleculas[coluna])
        if len(dados) == 0:
            dados = re.findall(r"([A-Z][a-z]*)(\d*)", moleculas[coluna])
        
        for atomo, linha in zip(atomos, range(len(atomos))):
            for i in dados:
                if i[0] == atomo:
                    if i[1] == "": 
                        coef[linha][coluna] += 1
                    else:
                        coef[linha][coluna] += int(i[1])
                
            if coluna >= qtd_reagentes:
                coef[linha][coluna] *= (-1)
                    

    return coef

In [8]:
def balaceamento():
    """
    Função que solicita uma equação química ao usuário e devolve a quantidade de 
    cada átomo de cada molécula, as moléculas e a matriz de coeficientes para balanceamento.

    Args:
        None

    Returns:
        tuple: Uma tupla contendo:
            - atomos (list of str): Lista de átomos únicos presentes na equação em ordem de ocorrência.
            - moleculas (list of str): Lista de moléculas (reagentes e produtos).
            - numpy.ndarray: Matriz de coeficientes para balanceamento da equação química.
    """
    
    print("Digite a equação para ser balanceada:")
    print("Siga o exemplo: Al + Cl2 -> AlCl3\n")
    formula = input("")
    formuladiv = formula.replace("+", "").partition("->")
    
    reagente = [i for i in formuladiv[0].split(" ") if i != ""]
    produto = [i for i in formuladiv[2].split(" ") if i != ""]
    atomos = encontra_atomos(reagente)

    A = encontra_coef(atomos, reagente+produto, len(reagente))
    b = np.zeros(A.shape[0])

    return calculo(A, b, formula)

### Output: Expressando o balanceamento encontrado

Essa é a última função a ser chamada e tem como objetivo apresentar os resultados. Para isso, é criada uma nova string que corresponde à equação fornecida no início do input, agora acompanhada dos coeficientes estequiométricos de cada molécula encontrados pelos métodos presentes nesse notebook.

In [9]:
def expressando(coeficientes, equacao):
    """
    Função que retorna a equação química balanceada como uma string, 
    combinando os coeficientes encontrados com o método numérico de Gauss-Seidel.

    Args:
        coeficientes (list or numpy.ndarray): Lista ou array contendo os coeficientes balanceados para as moléculas.
        equacao (str): String contendo a equação química no formato "reagente1 + reagente2 -> produto1 + produto2".

    Returns:
        str: A equação química balanceada com os coeficientes incluídos.
    """
    formuladiv = equacao.replace("+", "").partition("->")
    
    reagente = [i for i in formuladiv[0].split(" ") if i != ""]
    produto = [i for i in formuladiv[2].split(" ") if i != ""]
    moleculas = reagente + produto
    
    resultado = ""
    
    for i in range(len(reagente)):
        resultado += str(int(coeficientes[i])) + reagente[i] 
        if i != len(reagente) -1:
            resultado += " + "
    
    resultado += " -> "

    for j in range(len(produto)):
        resultado += str(int(coeficientes[j + len(reagente)])) + produto[j]
        if j != len(produto) -1:
            resultado += " + "

    return resultado

### Validação do Código

<p style="text-align:justify;">  Durante o processo de validação, é necessário testar tanto casos mais simples quanto aqueles que apresentam maior complexidade, como moléculas com átomos repetidos (por exemplo, NH₄NO₂) ou moléculas que utilizam parênteses e multiplicadores. O sucesso do código depende de sua capacidade de interpretar corretamente fórmulas químicas, identificar átomos únicos, processar moléculas com estruturas mais elaboradas e gerar uma matriz de coeficientes coerente. Com todas essas informações, o código é capaz de encontrar os coeficientes necessários para balancear a equação de maneira eficaz. Para validar o código de balanceamento de equações químicas e assegurar que ele funcione de maneira precisa e confiável em diferentes cenários, seguem alguns exemplos de equações balanceadas.

In [11]:
#Al + Cl2 -> AlCl3
equacao1 = balaceamento()
print("\nA equação balanceada:", equacao1)

Digite a equação para ser balanceada:
Siga o exemplo: Al + Cl2 -> AlCl3


A equação balanceada: 2Al + 3Cl2 -> 2AlCl3


In [ ]:
# NH4NO2 -> N2 + H2O
equacao2 = balaceamento()
print("\nA equação balanceada", equacao2)

Digite a equação para ser balanceada:
Siga o exemplo: Al + Cl2 -> AlCl3



 NH4NO2 -> N2 + H2O



A equação balanceada 1NH4NO2 -> 1N2 + 2H2O


In [ ]:
# Al + H2SO4 -> Al2(SO4)3 + H2
equacao3 = balaceamento()
print("\nA equação balanceada", equacao3)

Digite a equação para ser balanceada:
Siga o exemplo: Al + Cl2 -> AlCl3



 Al + H2SO4 -> Al2(SO4)3 + H2



A equação balanceada 2Al + 3H2SO4 -> 1Al2(SO4)3 + 3H2


In [ ]:
# Na + Cl -> NaCl
equacao5 = balaceamento()
print("\nA equação balanceada:", equacao5)

Digite a equação para ser balanceada:
Siga o exemplo: Al + Cl2 -> AlCl3



 Na + Cl -> NaCl



A equação balanceada: 1Na + 1Cl -> 1NaCl


In [ ]:
#C2H6O -> C4H10O + H2O
equacao6 = balaceamento()
print("\nA equação balanceada:", equacao6)

Digite a equação para ser balanceada:
Siga o exemplo: Al + Cl2 -> AlCl3



 C2H6O -> C4H10O + H2O



A equação balanceada: 2C2H6O -> 1C4H10O + 1H2O


In [ ]:
# NOH + H2 -> N2 + H2O
equacao4 = balaceamento()
print("\nA equação balanceada:", equacao4)

Digite a equação para ser balanceada:
Siga o exemplo: Al + Cl2 -> AlCl3



 NOH + H2 -> N2 + H2O



A equação balanceada: 2NOH + 1H2 -> 1N2 + 2H2O
